In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing #model_selection, 
from XGBoostPackage import xgbClass, modelfit
#from CrossValidation import CVScore
from sklearn.grid_search import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer

import xgboost as xgb
import datetime
from sklearn.metrics import mean_squared_error as mse, make_scorer
from sklearn.cross_validation import train_test_split
#now = datetime.datetime.now()

X_train = pd.read_csv('./input/train_further_additional_feature_filtered.csv',index_col=0)
y_train = pd.read_csv('./input/y_train_further_additional_feature_filtered.csv',index_col=0)['price_doc']
X_test = pd.read_csv('./input/test_further_additional_feature_filtered.csv',index_col=0)
macro = pd.read_csv('./input/macro.csv', parse_dates=['timestamp'])
id_test = pd.read_csv('./input/test.csv').id

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

#id_test = test.id
#multiplier = 0.969

def RMSLE_(y_val, y_val_pred):
    return np.sqrt(np.mean((np.log(y_val+1)-np.log(y_val_pred+1))**2))
RMSLE = make_scorer(RMSLE_, greater_is_better=False) 

def RMSE_(y_val, y_val_pred):
    return np.sqrt(np.mean((y_val-y_val_pred)**2))
RMSE = make_scorer(RMSE_, greater_is_better=False)

from time import time

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
imp = Imputer(missing_values=np.nan, strategy='median')
imp.fit(pd.concat([X_train,X_test]))
X_train = pd.DataFrame(imp.transform(X_train),columns=X_train.columns)
X_test = pd.DataFrame(imp.transform(X_test),columns=X_test.columns)

In [3]:
imp = Imputer(missing_values=np.infty, strategy='median')
imp.fit(pd.concat([X_train,X_test]))
X_train = pd.DataFrame(imp.transform(X_train),columns=X_train.columns)
X_test = pd.DataFrame(imp.transform(X_test),columns=X_test.columns)

In [ ]:
t0=time()
rfreg = RandomForestRegressor(max_depth=4, n_estimators=500, n_jobs=-1, random_state=123, max_features='auto')
param_grid = {
       'max_depth': np.arange(3,10,2),
       'max_features': [i/10.0 for i in np.arange(6,10)],
        'random_state':[123]#np.arange(20)
}
model = GridSearchCV(estimator=rfreg, param_grid=param_grid, n_jobs=-1, cv=5, scoring=RMSE)
model.fit(X_train, np.log(y_train+1))
print('Randon Forest Regressor...')
print('Best Params:')
print(model.grid_scores_, model.best_params_)
print('Best CV Score:')
print(-model.best_score_)
print((time()-t0)/60)

Process PoolWorker-9:
Process PoolWorker-11:
Process PoolWorker-12:
Process PoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 

In [13]:
rfreg = RandomForestRegressor(max_depth=6, n_estimators=500, n_jobs=-1, random_state=123, max_features=.95)
rfreg.fit(X_train, np.log(y_train+1))
y_predict = rfreg.predict(X_test)
y_predict = np.exp(y_predict)-1
#y_predict = np.round(y_predict)#np.round(y_predict * 0.99)
#output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})

In [14]:
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.to_csv('Jun16_5_RFTuning_maxD6_nEst500_maxFeatPt95_rndSt123.csv', index=False)